In [ ]:
import pandas as pd
import numpy as np  
import os
import gc
import psutil
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.signal import savgol_filter
from statsmodels.nonparametric.smoothers_lowess import lowess


In [2]:
# Get the current directory
current_dir = os.getcwd()

# Navigate one folder up
parent_dir = os.path.dirname(current_dir)

# Where the files are located
data = 'data\pull-pesquisas-city-2851556'

# Navigate down into the "data" folder
data_dir = os.path.join(parent_dir, data)


In [ ]:
# Initialize an empty list to store DataFrames
dataframes = []

# Variable to track total rows
total_rows = 0

# Loop through all files in the "data" folder
try:
    for file_name in os.listdir(data_dir):
        
        if file_name.endswith('.csv'):  # Check if the file is a CSV
            
            file_path = os.path.join(data_dir, file_name)
            
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            dataframes.append(df)  # Append the DataFrame to the list
            
             # Print dimensions of the current file
            print(f"File: {file_name} | Dimensions: {df.shape}")
            
            # Add the number of rows to the total count
            total_rows += df.shape[0]

    # Concatenate all DataFrames in the list by binding rows
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Print dimensions of the combined DataFrame
    print(f"Combined DataFrame Dimensions: {combined_df.shape}")

    # Verify the sum of rows matches
    if total_rows == combined_df.shape[0]:
        print("Row count verification successful! Total rows match.")
    else:
        print("Row count verification failed! Mismatch in row count.")

    print(combined_df.head())  # Display the first few rows of the combined DataFrame

except FileNotFoundError:
    print(f"Folder '{data_dir}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
print(combined_df.shape)

In [ ]:
# List the variables to delete manually
variables_to_delete = ['current_dir', 'parent_dir', 'data_dir', 'dataframes', 'file_name', 'file_path', 'df', 'total_rows', 'data', 'var', 'variables_to_delete']

# Iterate through the list and delete each variable
for var in variables_to_delete:
    if var in globals():  # Ensure the variable exists before trying to delete
        del globals()[var]

print("Memory cleared. Retained variables: combined_df")

In [ ]:
# Function to get memory usage in bytes
def memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss  # Resident Set Size (in bytes)

# Check memory usage before garbage collection
memory_before = memory_usage()

# Trigger garbage collection
gc.collect()

# Check memory usage after garbage collection
memory_after = memory_usage()

# Calculate memory cleared
memory_cleared = memory_before - memory_after

print(f"Memory before GC: {memory_before / 1024**2:.2f} MB")
print(f"Memory after GC: {memory_after / 1024**2:.2f} MB")
print(f"Memory cleared: {memory_cleared / 1024**2:.2f} MB")

In [ ]:
data_id_min = combined_df['Data_ID'].min()
data_id_max = combined_df['Data_ID'].max()
print(f"Min Data_ID: {data_id_min}, Max Data_ID: {data_id_max}")

In [ ]:
distinct_hotel_ids = combined_df['Hotel_ID'].nunique()
print(f"Number of distinct Hotel_IDs: {distinct_hotel_ids}")
del distinct_hotel_ids

In [ ]:
# Check memory usage before garbage collection
memory_before = memory_usage()

# Trigger garbage collection
gc.collect()

# Check memory usage after garbage collection
memory_after = memory_usage()

# Calculate memory cleared
memory_cleared = memory_before - memory_after

print(f"Memory before GC: {memory_before / 1024**2:.2f} MB")
print(f"Memory after GC: {memory_after / 1024**2:.2f} MB")
print(f"Memory cleared: {memory_cleared / 1024**2:.2f} MB")

In [ ]:
print(f"Number of distinct Hotel_IDs: {combined_df['Hotel_ID'].nunique()}")
print(f"Number of distinct Moeda_IDs: {combined_df['Moeda_ID'].nunique()}")
print(f"Number of distinct Canal_IDs: {combined_df['Canal_ID'].nunique()}")
print(f"Number of distinct Reservas: {combined_df['Reservas'].nunique()}")
print(f"Number of distinct DiariaMedia: {combined_df['DiariaMedia'].nunique()}")
print(f"Number of distinct Estadias: {combined_df['Estadia'].nunique()}")

In [ ]:
combined_df['Moeda_ID'].unique()

In [12]:
combined_df['DiariaMedia'] = combined_df.apply(
    lambda row: row['DiariaMedia'] * 0.16483969339817028 if row['Moeda_ID'] == 16 else row['DiariaMedia'], 
    axis=1
)

In [13]:
selected_columns_df = combined_df.loc[:, ['Data', 'Data_ID', 'Moeda_ID', 'Hotel_ID', 'Ocupacao_ID', 'DiariaMedia', "Estadia", 'Reservas']]

In [ ]:
print(f"Number of distinct Reservas: {selected_columns_df['Reservas'].nunique()}")

In [ ]:
print(selected_columns_df['Data_ID'].nunique())

In [ ]:
selected_columns_df.DiariaMedia.median()

In [ ]:
# Remove outliers at some point, but to be discussed
# selected_columns_df[selected_columns_df['DiariaMedia'] <= 1000] 


In [18]:
selected_columns_df = selected_columns_df.loc[:, ['Data', 'Data_ID',  'Hotel_ID', 'Ocupacao_ID', 'DiariaMedia', "Estadia", 'Reservas']]

In [ ]:
selected_columns_df.Reservas.sum() # 4925449

In [21]:
expanded_df = selected_columns_df.loc[selected_columns_df.index.repeat(selected_columns_df['Reservas'])].reset_index(drop=True)
expanded_df.Reservas = 1

In [ ]:
expanded_df.Reservas.sum() # 4925449 OK

In [ ]:
expanded_df.groupby('Hotel_ID').size().reset_index(name='Counts').sort_values(by='Counts', ascending=False)

In [ ]:
data_lake_prd_314410_cz_hoteis_competidores = pd.read_csv('c:\\Users\\paulo\\Desktop\\hotels brazil\\data\\lookups\\data-lake-prd-314410.cz.hoteis-competidores.csv')
data_lake_prd_314410_cz_hoteis_competidores[data_lake_prd_314410_cz_hoteis_competidores['Hotel_ID'] == 2094]['Competidor_ID']

In [72]:
hotel_df = expanded_df[expanded_df['DiariaMedia'] <= 1000]
hotel_df = hotel_df[hotel_df['Hotel_ID'].isin([2814, 3096, 3691, 2094])] 
hotel_df = hotel_df.dropna(subset=['DiariaMedia', 'Data'])
# Convert 'Data' to datetime format (if not already in datetime)
hotel_df['Data'] = pd.to_datetime(hotel_df['Data'])

# Sort the DataFrame by 'Data' to make sure the dates are in order for the fitting
hotel_df = hotel_df.sort_values(by='Data')


In [ ]:
hotel_df.groupby('Hotel_ID').size().reset_index(name='Counts')

In [ ]:
# Custom colors for each hotel
colors = ['red', 'blue','orange', 'purple']

# Create an empty figure to hold the multiple fitted curves
fig = px.line(title='LOESS Smoothed Fits of DiariaMedia Over Time for Selected Hotels')

# Loop through each hotel and create a smoothed fit for each one
for i, hotel_id in enumerate([2094,  2814, 3096, 3691]):
    # Filter data for the current hotel
    hotel_data = hotel_df[hotel_df['Hotel_ID'] == hotel_id]

     # Filter the data to only include dates from January 2022 onwards
    hotel_data['Data'] = hotel_data['Data'].dt.tz_localize(None)  # Remove timezone
    hotel_data = hotel_data[hotel_data['Data'] >= pd.to_datetime('2022-01-01')]

    # Get the values for fitting
    x_values = hotel_data['Data'].astype(np.int64) / 10**9  # Convert datetime to UNIX timestamp (seconds)
    y_values = hotel_data['DiariaMedia']

    # Apply LOESS smoothing (use frac for the smoothing parameter)
    smoothed_data = lowess(y_values, x_values, frac=0.1)  # Adjust frac for the level of smoothing (between 0 and 1)

    # Add the smoothed fit to the plot for the current hotel
    fig.add_scatter(x=hotel_data['Data'], y=smoothed_data[:, 1], mode='lines', name=f'Hotel_ID {hotel_id}', 
                    line=dict(color=colors[i], dash='dash'))

# Customize the layout for better appearance
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='DiariaMedia (USD)',
    template='plotly_dark',  # Optional: choose a theme
    xaxis=dict(showgrid=True, tickangle=45),  # Rotate x-axis labels for better readability
    yaxis=dict(showgrid=True),
)

# Show the interactive plot
fig.show()